In [1]:
from __future__ import print_function, division

from keras.datasets import mnist,fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys
import os
import numpy as np

class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = fashion_mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # labels = y_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise and generate a batch of new images
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator (real classified as ones and generated as zeros)
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator (wants discriminator to mistake images as real)
                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (
                epoch, global_step, d_loss[0], 100 * d_loss[1], g_loss))

                # If at save interval => save generated image samples
                if global_step % save_interval == 0:
                    self.save_imgs(epoch, global_step)

    def save_imgs(self, epoch, global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                cnt += 1
        if not os.path.isdir('images_dcgan_fashion_mnist'):
            os.mkdir('images_dcgan_fashion_mnist')
        fig.savefig("images_dcgan_fashion_mnist/epoch_%d_step_%d.png" % (epoch, global_step))
        plt.close()

if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=40, batch_size=64, save_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 1.076746, acc.: 35.16%] [G loss: 1.242499]
epoch:0 step:2 [D loss: 0.501441, acc.: 75.78%] [G loss: 1.628726]
epoch:0 step:3 [D loss: 0.362110, acc.: 85.16%] [G loss: 2.269358]
epoch:0 step:4 [D loss: 0.362636, acc.: 85.16%] [G loss: 2.107854]
epoch:0 step:5 [D loss: 0.368652, acc.: 85.94%] [G loss: 1.867027]
epoch:0 step:6 [D loss: 0.263538, acc.: 89.06%] [G loss: 1.976107]
epoch:0 step:7 [D loss: 0.213480, acc.: 95.31%] [G loss: 1.763083]
epoch:0 step:8 [D loss: 0.369741, acc.: 85.94%] [G loss: 1.252625]
epoch:0 step:9 [D loss: 0.377612, acc.: 83.59%] [G loss: 1.541014]
epoch:0 step:10 [D loss: 0.379740, acc.: 82.03%] [G loss: 1.668240]
epoch:0 step:11 [D loss: 0.623777, acc.: 70.31%] [G loss: 1.823322]
epoch:0 step:12 [D loss: 0.613963, acc.: 67.97%] [G loss: 1.782833]
epoch:0 step:13 [D loss: 0.772434, acc.: 57.03%] [G loss: 1.759863]
epoch:0 step:14 [D loss: 0.742470, acc.: 60.94%] [G loss: 1.451238]
epoch:0 step:15 [D loss: 0.902387, acc.: 50.00%] [G loss:

epoch:0 step:126 [D loss: 0.707632, acc.: 57.03%] [G loss: 1.251627]
epoch:0 step:127 [D loss: 0.736849, acc.: 60.16%] [G loss: 1.215515]
epoch:0 step:128 [D loss: 0.753354, acc.: 53.91%] [G loss: 1.258323]
epoch:0 step:129 [D loss: 0.748187, acc.: 53.91%] [G loss: 1.083571]
epoch:0 step:130 [D loss: 0.675359, acc.: 57.81%] [G loss: 1.117549]
epoch:0 step:131 [D loss: 0.781838, acc.: 55.47%] [G loss: 1.207138]
epoch:0 step:132 [D loss: 0.664388, acc.: 62.50%] [G loss: 1.167876]
epoch:0 step:133 [D loss: 0.704462, acc.: 59.38%] [G loss: 1.188114]
epoch:0 step:134 [D loss: 0.656916, acc.: 64.84%] [G loss: 1.038236]
epoch:0 step:135 [D loss: 0.670578, acc.: 61.72%] [G loss: 1.268355]
epoch:0 step:136 [D loss: 0.764708, acc.: 56.25%] [G loss: 0.983205]
epoch:0 step:137 [D loss: 0.684785, acc.: 65.62%] [G loss: 1.217371]
epoch:0 step:138 [D loss: 0.749515, acc.: 54.69%] [G loss: 1.323973]
epoch:0 step:139 [D loss: 0.823349, acc.: 45.31%] [G loss: 1.215836]
epoch:0 step:140 [D loss: 0.702694

...

epoch:39 step:37001 [D loss: 0.441726, acc.: 78.91%] [G loss: 1.444735]
epoch:39 step:37002 [D loss: 0.512050, acc.: 73.44%] [G loss: 1.624398]
epoch:39 step:37003 [D loss: 0.656588, acc.: 60.94%] [G loss: 1.672837]
epoch:39 step:37004 [D loss: 0.802401, acc.: 52.34%] [G loss: 1.829042]
epoch:39 step:37005 [D loss: 0.421984, acc.: 82.81%] [G loss: 1.963761]
epoch:39 step:37006 [D loss: 0.555894, acc.: 74.22%] [G loss: 1.799433]
epoch:39 step:37007 [D loss: 0.694801, acc.: 58.59%] [G loss: 1.210439]
epoch:39 step:37008 [D loss: 0.512552, acc.: 78.12%] [G loss: 1.284373]
epoch:39 step:37009 [D loss: 0.497359, acc.: 82.03%] [G loss: 1.620774]
epoch:39 step:37010 [D loss: 0.262752, acc.: 94.53%] [G loss: 1.695931]
epoch:39 step:37011 [D loss: 0.557161, acc.: 71.88%] [G loss: 1.595615]
epoch:39 step:37012 [D loss: 0.476878, acc.: 77.34%] [G loss: 1.562918]
epoch:39 step:37013 [D loss: 0.817231, acc.: 54.69%] [G loss: 1.308623]
epoch:39 step:37014 [D loss: 0.644730, acc.: 65.62%] [G loss: 1.

epoch:39 step:37116 [D loss: 0.588284, acc.: 71.09%] [G loss: 1.400137]
epoch:39 step:37117 [D loss: 0.549016, acc.: 68.75%] [G loss: 1.564304]
epoch:39 step:37118 [D loss: 0.621899, acc.: 60.16%] [G loss: 1.847567]
epoch:39 step:37119 [D loss: 0.530718, acc.: 77.34%] [G loss: 1.541503]
epoch:39 step:37120 [D loss: 0.406718, acc.: 82.03%] [G loss: 1.654928]
epoch:39 step:37121 [D loss: 0.643065, acc.: 64.06%] [G loss: 1.282339]
epoch:39 step:37122 [D loss: 0.630184, acc.: 67.97%] [G loss: 1.776597]
epoch:39 step:37123 [D loss: 0.420835, acc.: 78.91%] [G loss: 2.110442]
epoch:39 step:37124 [D loss: 0.626780, acc.: 65.62%] [G loss: 1.630700]
epoch:39 step:37125 [D loss: 0.415929, acc.: 82.03%] [G loss: 1.284189]
epoch:39 step:37126 [D loss: 0.522410, acc.: 73.44%] [G loss: 1.425661]
epoch:39 step:37127 [D loss: 0.471805, acc.: 82.03%] [G loss: 1.549665]
epoch:39 step:37128 [D loss: 0.729196, acc.: 49.22%] [G loss: 1.278283]
epoch:39 step:37129 [D loss: 0.335351, acc.: 91.41%] [G loss: 1.

epoch:39 step:37231 [D loss: 0.627104, acc.: 65.62%] [G loss: 1.851865]
epoch:39 step:37232 [D loss: 0.605644, acc.: 64.06%] [G loss: 1.525447]
epoch:39 step:37233 [D loss: 0.428133, acc.: 85.16%] [G loss: 1.567205]
epoch:39 step:37234 [D loss: 0.508339, acc.: 74.22%] [G loss: 2.199122]
epoch:39 step:37235 [D loss: 0.581085, acc.: 75.00%] [G loss: 1.448476]
epoch:39 step:37236 [D loss: 0.588172, acc.: 68.75%] [G loss: 1.143348]
epoch:39 step:37237 [D loss: 0.638150, acc.: 72.66%] [G loss: 1.601812]
epoch:39 step:37238 [D loss: 0.582552, acc.: 69.53%] [G loss: 1.854965]
epoch:39 step:37239 [D loss: 0.684250, acc.: 62.50%] [G loss: 1.722899]
epoch:39 step:37240 [D loss: 0.576179, acc.: 73.44%] [G loss: 1.319005]
epoch:39 step:37241 [D loss: 0.679353, acc.: 60.94%] [G loss: 1.491600]
epoch:39 step:37242 [D loss: 0.540792, acc.: 67.97%] [G loss: 1.611903]
epoch:39 step:37243 [D loss: 0.378597, acc.: 85.94%] [G loss: 2.246260]
epoch:39 step:37244 [D loss: 0.324755, acc.: 85.16%] [G loss: 1.

epoch:39 step:37346 [D loss: 0.560347, acc.: 71.88%] [G loss: 1.416011]
epoch:39 step:37347 [D loss: 0.458682, acc.: 76.56%] [G loss: 1.790535]
epoch:39 step:37348 [D loss: 0.590114, acc.: 68.75%] [G loss: 1.385464]
epoch:39 step:37349 [D loss: 0.817984, acc.: 57.03%] [G loss: 2.037214]
epoch:39 step:37350 [D loss: 0.632149, acc.: 68.75%] [G loss: 1.400572]
epoch:39 step:37351 [D loss: 0.347501, acc.: 86.72%] [G loss: 1.726753]
epoch:39 step:37352 [D loss: 0.471568, acc.: 78.12%] [G loss: 1.537224]
epoch:39 step:37353 [D loss: 0.520833, acc.: 74.22%] [G loss: 0.897506]
epoch:39 step:37354 [D loss: 0.435373, acc.: 82.81%] [G loss: 1.846293]
epoch:39 step:37355 [D loss: 0.401537, acc.: 80.47%] [G loss: 1.172756]
epoch:39 step:37356 [D loss: 0.515017, acc.: 76.56%] [G loss: 1.620640]
epoch:39 step:37357 [D loss: 0.409567, acc.: 85.16%] [G loss: 1.732549]
epoch:39 step:37358 [D loss: 0.652946, acc.: 64.06%] [G loss: 1.253320]
epoch:39 step:37359 [D loss: 0.351415, acc.: 88.28%] [G loss: 1.

epoch:39 step:37461 [D loss: 0.587288, acc.: 71.88%] [G loss: 1.165016]
epoch:39 step:37462 [D loss: 0.322861, acc.: 88.28%] [G loss: 1.694633]
epoch:39 step:37463 [D loss: 0.512190, acc.: 71.09%] [G loss: 2.003757]
epoch:39 step:37464 [D loss: 0.602923, acc.: 72.66%] [G loss: 1.150062]
epoch:39 step:37465 [D loss: 0.727918, acc.: 53.12%] [G loss: 1.358097]
epoch:39 step:37466 [D loss: 0.707430, acc.: 62.50%] [G loss: 1.983932]
epoch:39 step:37467 [D loss: 0.477298, acc.: 76.56%] [G loss: 1.469891]
epoch:39 step:37468 [D loss: 0.679826, acc.: 64.06%] [G loss: 0.930553]
epoch:39 step:37469 [D loss: 0.464617, acc.: 84.38%] [G loss: 1.559571]
epoch:39 step:37470 [D loss: 0.669093, acc.: 64.84%] [G loss: 1.249963]
epoch:39 step:37471 [D loss: 0.558440, acc.: 71.09%] [G loss: 1.282879]
epoch:39 step:37472 [D loss: 0.343345, acc.: 88.28%] [G loss: 1.636791]
epoch:39 step:37473 [D loss: 0.714859, acc.: 62.50%] [G loss: 1.454905]
epoch:39 step:37474 [D loss: 0.476055, acc.: 78.91%] [G loss: 1.